In [1]:
# -*- coding: utf-8 -*-
"""
اختبار نموذج STT بصوتك الخاص
"""

import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import json
import pickle
import sounddevice as sd
from scipy.io import wavfile
import IPython.display as ipd

print("="*80)
print("🎤 اختبار نموذج التعرف على الكلام بصوتك")
print("="*80)

# ============================================================================
# 1. تحميل النموذج والإعدادات المحفوظة
# ============================================================================
print("\n📥 تحميل النموذج والإعدادات...")

# تحميل النموذج
model = keras.models.load_model('improved_stt_model.keras')
print("✅ تم تحميل النموذج")

# تحميل الإعدادات
with open('improved_model_config.json', 'r') as f:
    config = json.load(f)
print("✅ تم تحميل الإعدادات")

# تحميل المعالجات
with open('improved_preprocessors.pkl', 'rb') as f:
    preprocessors = pickle.load(f)
    char_to_num = preprocessors['char_to_num']
    num_to_char = preprocessors['num_to_char']
print("✅ تم تحميل المعالجات")

# استخراج الإعدادات
frame_length = config['frame_length']
frame_step = config['frame_step']
fft_length = config['fft_length']

print(f"\n⚙️ إعدادات النموذج:")
print(f"  - Frame Length: {frame_length}")
print(f"  - Frame Step: {frame_step}")
print(f"  - FFT Length: {fft_length}")

# ============================================================================
# 2. دوال المعالجة والتنبؤ
# ============================================================================

def process_audio_file(audio_path):
    """معالجة ملف صوتي وتحويله لسبيكتروجرام"""
    try:
        # قراءة الملف
        audio = tf.io.read_file(audio_path)
        audio, sample_rate = tf.audio.decode_wav(audio)
        audio = tf.squeeze(audio, axis=-1)
        audio = tf.cast(audio, tf.float32)
        
        print(f"  📊 معدل العينات: {sample_rate.numpy()} Hz")
        print(f"  ⏱️ مدة التسجيل: {len(audio)/sample_rate.numpy():.2f} ثانية")
        
        # استخراج السبيكتروجرام
        spectrogram = tf.signal.stft(
            audio,
            frame_length=frame_length,
            frame_step=frame_step,
            fft_length=fft_length
        )
        spectrogram = tf.abs(spectrogram)
        spectrogram = tf.math.pow(spectrogram, 0.5)
        
        # تطبيع
        means = tf.math.reduce_mean(spectrogram, 1, keepdims=True)
        stddevs = tf.math.reduce_std(spectrogram, 1, keepdims=True)
        spectrogram = (spectrogram - means) / (stddevs + 1e-10)
        
        return spectrogram, sample_rate.numpy()
    
    except Exception as e:
        print(f"❌ خطأ في معالجة الملف: {e}")
        return None, None


def decode_prediction(pred, temperature=0.8):
    """فك تشفير تنبؤ النموذج"""
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    
    # تطبيق temperature
    if temperature != 1.0:
        pred = pred / temperature
        pred = tf.nn.softmax(pred, axis=-1)
    
    # Greedy decoding
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0]
    
    # تحويل لنص
    output_text = []
    for result in results:
        chars = num_to_char(result)
        text = tf.strings.reduce_join(chars).numpy().decode("utf-8")
        text = ' '.join(text.split()).strip()
        output_text.append(text)
    
    return output_text[0] if output_text else ""


def predict_from_audio(audio_path, show_alternatives=True):
    """التنبؤ بالنص من ملف صوتي"""
    print(f"\n🔍 معالجة الملف: {audio_path}")
    
    # معالجة الصوت
    spectrogram, sample_rate = process_audio_file(audio_path)
    if spectrogram is None:
        return None
    
    # إضافة بُعد الدفعة
    spectrogram = tf.expand_dims(spectrogram, axis=0)
    
    # التنبؤ
    print("  🤖 جاري التنبؤ...")
    prediction = model(spectrogram, training=False)
    
    # فك التشفير مع درجات حرارة مختلفة
    if show_alternatives:
        print("\n  📝 التنبؤات بدرجات حرارة مختلفة:")
        temperatures = [0.5, 0.8, 1.0, 1.2]
        predictions = []
        
        for temp in temperatures:
            text = decode_prediction(prediction, temperature=temp)
            predictions.append(text)
            print(f"    Temperature {temp}: {text}")
        
        # اختيار أفضل تنبؤ (الأطول عادة)
        best_prediction = max(predictions, key=len)
        return best_prediction
    else:
        return decode_prediction(prediction, temperature=0.8)

# ============================================================================
# 3. تسجيل الصوت
# ============================================================================

def record_audio(duration=5, sample_rate=16000, filename="my_recording.wav"):
    """تسجيل الصوت من الميكروفون"""
    print(f"\n🎙️ جاري التسجيل لمدة {duration} ثانية...")
    print("   تكلم الآن! 🗣️")
    
    try:
        # التسجيل
        recording = sd.rec(
            int(duration * sample_rate),
            samplerate=sample_rate,
            channels=1,
            dtype='float32'
        )
        sd.wait()  # انتظار انتهاء التسجيل
        
        print("   ✅ تم التسجيل!")
        
        # حفظ الملف
        wavfile.write(filename, sample_rate, recording)
        print(f"   💾 تم حفظ التسجيل في: {filename}")
        
        return filename
    
    except Exception as e:
        print(f"   ❌ خطأ في التسجيل: {e}")
        print("   💡 تأكد من توصيل الميكروفون!")
        return None

# ============================================================================
# 4. الواجهة الرئيسية
# ============================================================================

def test_with_recorded_audio():
    """تسجيل واختبار الصوت"""
    print("\n" + "="*80)
    print("🎤 تسجيل صوتك واختباره")
    print("="*80)
    
    # اختيار مدة التسجيل
    print("\n⏱️ اختر مدة التسجيل:")
    print("  1. 3 ثواني (جملة قصيرة)")
    print("  2. 5 ثواني (جملة متوسطة)")
    print("  3. 10 ثواني (جملة طويلة)")
    
    duration_map = {'1': 3, '2': 5, '3': 10}
    choice = input("\nاختيارك (1-3): ").strip()
    duration = duration_map.get(choice, 5)
    
    # تسجيل الصوت
    filename = record_audio(duration=duration)
    
    if filename:
        # تشغيل التسجيل
        print("\n🔊 تشغيل التسجيل...")
        display(ipd.Audio(filename))
        
        # التنبؤ
        prediction = predict_from_audio(filename, show_alternatives=True)
        
        print("\n" + "="*80)
        print("🎯 النتيجة النهائية:")
        print("="*80)
        print(f"📝 النص المتوقع: {prediction}")
        print("="*80)
        
        return prediction
    else:
        print("\n❌ فشل التسجيل!")
        return None


def test_with_existing_file():
    """اختبار ملف صوتي موجود"""
    print("\n" + "="*80)
    print("📁 اختبار ملف صوتي موجود")
    print("="*80)
    
    audio_path = input("\n🎵 أدخل مسار الملف الصوتي (.wav): ").strip()
    
    if not os.path.exists(audio_path):
        print(f"❌ الملف غير موجود: {audio_path}")
        return None
    
    # تشغيل الملف
    print("\n🔊 تشغيل الملف...")
    display(ipd.Audio(audio_path))
    
    # التنبؤ
    prediction = predict_from_audio(audio_path, show_alternatives=True)
    
    print("\n" + "="*80)
    print("🎯 النتيجة النهائية:")
    print("="*80)
    print(f"📝 النص المتوقع: {prediction}")
    print("="*80)
    
    return prediction


def batch_test_files():
    """اختبار عدة ملفات دفعة واحدة"""
    print("\n" + "="*80)
    print("📦 اختبار ملفات متعددة")
    print("="*80)
    
    folder_path = input("\n📂 أدخل مسار المجلد الذي يحتوي على الملفات: ").strip()
    
    if not os.path.exists(folder_path):
        print(f"❌ المجلد غير موجود: {folder_path}")
        return
    
    # البحث عن ملفات wav
    wav_files = [f for f in os.listdir(folder_path) if f.endswith('.wav')]
    
    if not wav_files:
        print("❌ لا توجد ملفات .wav في المجلد!")
        return
    
    print(f"\n✅ تم العثور على {len(wav_files)} ملف")
    
    results = []
    for i, wav_file in enumerate(wav_files[:10], 1):  # أول 10 ملفات فقط
        print(f"\n{'='*60}")
        print(f"ملف {i}/{min(10, len(wav_files))}: {wav_file}")
        print('='*60)
        
        audio_path = os.path.join(folder_path, wav_file)
        prediction = predict_from_audio(audio_path, show_alternatives=False)
        
        results.append({
            'file': wav_file,
            'prediction': prediction
        })
        
        print(f"📝 النتيجة: {prediction}")
    
    # عرض ملخص النتائج
    print("\n" + "="*80)
    print("📊 ملخص النتائج")
    print("="*80)
    for i, result in enumerate(results, 1):
        print(f"{i}. {result['file']}")
        print(f"   → {result['prediction']}\n")


# ============================================================================
# 5. القائمة الرئيسية
# ============================================================================

def main_menu():
    """القائمة الرئيسية"""
    while True:
        print("\n" + "="*80)
        print("🎤 قائمة اختبار نموذج التعرف على الكلام")
        print("="*80)
        print("\n📋 الخيارات المتاحة:")
        print("  1. 🎙️  تسجيل صوتي واختباره")
        print("  2. 📁 اختبار ملف صوتي موجود")
        print("  3. 📦 اختبار عدة ملفات دفعة واحدة")
        print("  4. 🚪 خروج")
        
        choice = input("\n✨ اختر رقماً (1-4): ").strip()
        
        if choice == '1':
            test_with_recorded_audio()
        elif choice == '2':
            test_with_existing_file()
        elif choice == '3':
            batch_test_files()
        elif choice == '4':
            print("\n👋 شكراً لاستخدامك البرنامج!")
            break
        else:
            print("\n❌ اختيار غير صحيح! حاول مرة أخرى.")
        
        input("\n⏸️  اضغط Enter للمتابعة...")


# ============================================================================
# 6. تشغيل البرنامج
# ============================================================================

if __name__ == "__main__":
    try:
        main_menu()
    except KeyboardInterrupt:
        print("\n\n🛑 تم إيقاف البرنامج")
    except Exception as e:
        print(f"\n❌ خطأ: {e}")
        import traceback
        traceback.print_exc()

ModuleNotFoundError: No module named 'numpy'

In [2]:
!pip install numpy 

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.8 MB 4.2 MB/s eta 0:00:03
   ---- ----------------------------------- 1.3/12.8 MB 3.9 MB/s eta 0:00:03
   ------ --------------------------------- 2.1/12.8 MB 3.8 MB/s eta 0:00:03
   --------- ------------------------------ 2.9/12.8 MB 3.8 MB/s eta 0:00:03
   ----------- ---------------------------- 3.7/12.8 MB 3.8 MB/s eta 0:00:03
   ------------- -------------------------- 4.5/12.8 MB 3.8 MB/s eta 0:00:03
   ----------------- ---------------------- 5.5/12.8 MB 3.9 MB/s eta 0:00:02
   ------------------- -------------------- 6.3/12.8 MB 3.8 MB/s eta 0:00:02
   --------------------- ------------------ 6.8/12.8 MB 3.8 MB/s eta 0:00:02
   ----------------------- ---------------- 7.6/12.8 MB 3.8 MB/s eta 0:00:02
   -------------------------- ------------- 8.4/12.8 MB 3.7 MB/s eta 0:00:02
   --

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\acer\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.20.0-cp312-cp312-win_amd64.whl.metadata (4.6 kB)
  Using cached librosa-0.11.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached PyAudio-0.2.14-cp312-cp312-win_amd64.whl.metadata (2.7 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached tenso

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\acer\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\